2025-10-02

# 学習内容
### 書籍/教材: スッキリわかるPythonによる機械学習入門
### 範囲: 第5章 分類１：アヤメの判別
### 目的: より複雑な分類の機械学習を行う。決定木について学ぶ。


 -----------------------------
## <内容まとめ>

・uniqueメソッドで種類列の値を確認
uniqueメソッドは重複を除いた値を返す。
```python
df['種類'].unique()
```

・array型の特定要素を参照
```python
syurui = df['種類'].unique()
syurui[0]
```

・value_countsメソッドでデータの出現回数をカウント
```python
df['種類'].value_counts()
```
df.mean()
・欠損値
iris.csvの[75行目,がく辺幅]のように空欄で欠損している値を**欠損値**という。

・isnullメソッドで欠損値の有無を調べる
isnullメソッドはデータフレームの各セルを調べてNaNならTrueを返す。
```python
df.isnull()
```

・anyメソッドにより列単位で欠損値があるか確認
anyメソッドは各列にたいして一つ以上の欠損値がある場合Trueを返す。
```python
df.isnull().any(axis=0)
```

・sumメソッドで各列の合計値を確認
```python
df.sum
```

・sumメソッドとisnullメソッドで各列の欠損値の数を調べる
以下ではtmpにTrueとFalseだけで構成されたデータフレーム(df.isnull())をtmpに代入して、列ごとの欠損値の合計を求めている。
```python
tmp = df.isnull()
tmp.sum()
# 以下のように書くことも可能
df.isnull().sum()
```

・dropnaメソッドで欠損値のある行/列を削除する
dropnaメソッドは欠損値が削除されたデータフレームを返り値として返す。元のデータフレームには変更を加えない(dfは欠損値があるまま)。
```python
df2 = df.dropna(how = 'any', axis = 0)
# axis = 1とすると欠損値がある列を削除
# axis = 0とすると欠損値がある行を削除
# how = 'all'とするとすべてが欠損値となっている行または列を削除
# how = 'any'とするとどれか一つが欠損値となっている行または列を削除
# axis = 0 の後に inplace = True を加えるとdf自体を変更する
```

・fillnaメソッドで欠損値を指定した値に置き換える
fillnaメソッドは欠損値を指定した値で書き換えたデータフレームを新たに作成し、そのデータフレームを返り値として返す。
```python
df['花弁長さ'] = df['花弁長さ'].fillna(0)
# .fillna(0)で花弁の長さの欠損値を0に置き換えている
```

・平均値の計算
今回は欠損値を平均値で置き換える。
```python
#numeric_only引数をTrueにすることで、文字列の列(今回の場合は種類列)を除外して計算してくれる
df.mean(numeric_only=True)
```

・特定の列だけ計算
```python
#花弁長さ列の平均値を計算する
df['花弁長さ'].mean()
```

・基本統計量をもとめるメソッド一覧
```
平均値 - mean
分散 - var
標準偏差 - std
中央値 - median
合計 - sum 
最大値 - max
最小値 - min
最頻値 - mode
```

・決定木モデル
決定木モデルは条件分岐をひたすら繰り返す手法。どのような条件にするかを機械学習を用いて設定する。そのとき、その条件分岐が良い分類か悪い分類かを判断する基準として、子ノード内の正解データの比率を示す**不純度**という数値的指標がある。
すなわち、条件分岐として考えらえる様々な条件と、その分岐後の二つの子ノードの不純度の値をすべて計算（scikit learnでは時間削減のため一部計算）し、最も不純度が小さくなるような条件分岐を良い条件とみなす。

・モデルの作成
max_depthには決定木の深さの最大値を指定できる。木を深くするほど条件分岐の回数も増えるため、正解率が高くなる可能性が高くなる。しかし、視覚的な分かりやすさは失われる。
random_stateは乱数のシードを指定する。これは乱数を使うときの基準となる値で、同じシードを指定すれば毎回同じ乱数が順に得られる。すなわち再現性を持たせることが出来る。(決定木モデルでは乱数を使用するため、毎回異なる学習結果になってしまう。)
```python
# 関数のインポート
from sklearn import tree
# モデルの作成
model = tree.DecisionTreeClassifier(max_depth = 2,
                                    random_state=0)
```

・ホールドアウト法
モデルを評価したい場合、学習に使用したデータに対し正確な回答ができるのは当たり前。よって訓練データ（学習に使用したデータ）とは別にテストデータ（正解率の計算のみに使用するデータ）がほしい。よって教師データを訓練データとテストデータに分割する方法を**ホールドアウト法**という。scikit learnではtrain_test_split関数を使う。
```python
# 関数のインポート
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, t,test_size = 0.3, random_state = 0)
#x_train,y_trainが学習に利用する訓練データ
#x_test,y_testが検証に利用するテストデータ
```
ここでtrain_test_split関数の引数のtest_sizeとrandom_stateを指定している。  
test_sizeは全体のデータに対するテストデータの割合を0.0~1.0で指定する。
random_stateでは乱数のシードを指定している。(再現性を持たせる)

・分岐条件の列を決める
列番号は0から順に振られる。分岐条件が-2となっている場合は、そのノードはそれ以上分岐しないことを示す。
```python
model.tree_.feature
```

・分岐条件のしきい値
しきい値とは条件が分岐する境目の値。
tree_.thresholdは分岐条件のしきい値を含む配列を返す。
```python
model.tree_.threshold
```

・末端ノード（リーフ）と種類の紐づけ
```python
# ノード番号1、3、4に到達したアヤメの種類ごとの数
print(model.tree_.value[1]) # ノード番号1に到達したとき
print(model.tree_.value[3]) # ノード番号3に到達したとき
print(model.tree_.value[4]) # ノード番号4に到達したとき
```
```python
model.classes_
```

分岐条件に対応する列、しきい値、リーフと種類の紐づけより、決定木のフローチャートを描くことができる。

 -----------------------------
## まとめ



--------------------------------------
## 以下は実装例



In [4]:
import pandas as pd

In [5]:
df = pd.read_csv('chap05/iris.csv')

In [6]:
#種類列の値を確認
df['種類'].unique()

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

In [7]:
#array型の特定要素を参照
syurui = df['種類'].unique()
syurui[0:2]

array(['Iris-setosa', 'Iris-versicolor'], dtype=object)

In [8]:
#出現回数のカウント
df['種類'].value_counts()

種類
Iris-setosa        50
Iris-versicolor    50
Iris-virginica     50
Name: count, dtype: int64

In [9]:
df.isnull()#各マスが欠損値かどうか調べる

,がく片長さ,がく片幅,花弁長さ,花弁幅,種類
0,False,False,False,False,False
1,False,False,False,False,False
2,False,False,False,False,False
3,False,False,False,False,False
4,False,False,False,False,False
...,...,...,...,...,...
145,False,False,False,False,False
146,False,False,False,False,False
147,False,False,True,False,False
148,False,False,False,False,False


In [10]:
#各列に欠損値があるか確認
df.isnull().any(axis=0)
#以下の結果より、四つの列に一つ以上欠損値があるということが分かる

がく片長さ     True
がく片幅      True
花弁長さ      True
花弁幅       True
種類       False
dtype: bool

In [11]:
# 各列の合計を確認
df.sum()

がく片長さ                                                62.29
がく片幅                                                 65.62
花弁長さ                                                 72.04
花弁幅                                                  66.22
種類       Iris-setosaIris-setosaIris-setosaIris-setosaIr...
dtype: object

In [12]:
# 各列の欠損値の数を調べる
tmp = df.isnull()
tmp.sum()

がく片長さ    2
がく片幅     1
花弁長さ     2
花弁幅      2
種類       0
dtype: int64

In [13]:
# 欠損値が1つでもある行を削除した結果を、df2に代入
df2 = df.dropna(how = 'any', axis = 0)

df2.tail(3) # 欠損値の存在確認
#以下の結果を見れば欠損値のあった147行目が削除されていることがわかる

,がく片長さ,がく片幅,花弁長さ,花弁幅,種類
146,0.56,0.21,0.69,0.46,Iris-virginica
148,0.53,0.58,0.63,0.92,Iris-virginica
149,0.44,0.42,0.41,0.71,Iris-virginica


In [14]:
# 花弁の長さの欠損値を0に置き換える
df['花弁長さ'] = df['花弁長さ'].fillna(0)
df.tail(3)
# 下の結果より147行目の花弁の長さが0に置き換えられていることがわかる

,がく片長さ,がく片幅,花弁長さ,花弁幅,種類
147,0.61,0.42,0.00,0.79,Iris-virginica
148,0.53,0.58,0.63,0.92,Iris-virginica
149,0.44,0.42,0.41,0.71,Iris-virginica


In [15]:
#各行の平均値をもとめる。
df.mean(numeric_only=True)

がく片長さ    0.420878
がく片幅     0.440403
花弁長さ     0.480267
花弁幅      0.447432
dtype: float64

In [16]:
#がく片長さ列の平均値を計算
df['がく片長さ'].mean()

0.42087837837837844

In [17]:
df = pd.read_csv('chap05/iris.csv')

# 各列の平均値を計算して、colmeanに代入
colmean = df.mean(numeric_only=True)

# 平均値で欠損値を穴埋めしてdf2に代入
df2 = df.fillna(colmean)
# 欠損値があるか確認
df2.isnull().any(axis = 0)

がく片長さ    False
がく片幅     False
花弁長さ     False
花弁幅      False
種類       False
dtype: bool

In [22]:
xcol = ['がく片長さ','がく片幅', '花弁長さ', '花弁幅']
x= df2[xcol]
t = df2['種類']

In [23]:
# 関数のインポート
from sklearn import tree
# モデルの作成
model = tree.DecisionTreeClassifier(max_depth = 2,
                                    random_state=0)

In [25]:
# ホールドアウト法で訓練データとテストデータに分割
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, t,test_size = 0.3, random_state = 0)
# test_size = 0.3なので全体のデータの３割がテストデータとして使用される。

In [26]:
# 訓練データで再学習
model.fit(x_train, y_train)

# テストデータの予測結果と実際の答えが合致する正解率を計算
model.score(x_test, y_test)

0.9555555555555556

In [27]:
import pickle
with open('chap05/irismodel.pkl', 'wb') as f:
    pickle.dump(model, f)